In [11]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np
from itertools import product
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from scipy import stats

###############################################################################################
path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/'
path_df = join(path,'Bilan_classif/2_signif_elecs_patients/')
path2save = join(path,'Bilan_classif/3_Figures/RL_sep/')
###############################################################################################

freqs = {'2_theta':'win3','6_gamma2':'win1'}#['1_delta','2_theta','3_alpha','4_beta','5_gamma1','6_gamma2']#
conds = {'0':['poor','partial',5],'1':['partial','detailed',4],
     '2':['poor','detailed',4]}
feats = ['da','pow_change']

#Create the list of rois to create the global matrix
for cond in conds:
    for freq in freqs:
        filename = path_df+cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq+'_'+freqs[freq]+'_patients'+str(conds[cond][2])+'.csv'
        df = pd.read_csv(filename)
        rois = df['s_aal'].values
        roi_all = np.unique(rois)
        roi_all = np.delete(roi_all,np.where(roi_all=='Not found'), axis=0)
        df['pow_change']= (df['p1']-df['p0'])/df['p0']
        for roi in roi_all:
            gr = df.groupby(['s_aal','su_codes'])
            sel = gr[feats].agg(('max','min','mean'))
            df_roi = sel[sel.index.get_level_values(0)==roi]
            da_val = df_roi.filter(like='da').filter(like='max').values
            da, da_std = np.mean(da_val), stats.sem(da_val)
            pow_min = df_roi.filter(like='pow_change').filter(like='min').values
            pow_max = df_roi.filter(like='pow_change').filter(like='max').values
            rel_change_val = []
            for v in range(len(pow_min)):
                if abs(pow_min[v])>abs(pow_max[v]):
                    rel_change_val.append(pow_min[v][0])
                elif abs(pow_min[v])<=abs(pow_max[v]):
                    rel_change_val.append(pow_max[v][0])
            change_pow, change_std = np.mean(rel_change_val), stats.sem(rel_change_val)
            print(roi,change_pow,change_std,da)
            #Plot all information by ROI for all subjects
            xfmt = ScalarFormatter(useMathText=True)
            #xfmt.set_powerlimits((0,3))
            fig = plt.figure(1,figsize=(5,5))
            title = roi+' \n '+conds[cond][0].capitalize()+' vs '+conds[cond][1].capitalize()+' '+freq[2:].capitalize()
            fig.suptitle(title, fontsize=14)

            ax = fig.add_subplot(111)
            ax2 = ax.twinx()
            pow_err = np.array([(change_std,0)]).T
            da_err = np.array([(da_std,0)]).T
            if change_pow>0:
                ax.bar([0.2], change_pow, align='center',yerr=pow_err,width=0.2,color='red',label='Power change')
            else:
                ax.bar([0.2], change_pow, align='center',yerr=pow_err,width=0.2,color='b',label='Power change')
            if freq == '2_theta':
                ax.set_ylim(-3.5,3.5)
            elif freq == '6_gamma2':
                ax.set_ylim(-0.8,0.8)
            ax2.bar([0.5], da, align='center',yerr=da_err,width=0.2,color='orange',label='AUC Score')

            ax.set_ylabel('Power change (Z-score)',fontsize=11)
            ax2.set_ylim(0,1)
            
            ax.set_xticklabels('')
            ax.tick_params(axis='x', which='both', bottom='off',top='off',labelbottom='off')
            ax2.set_ylabel('AUC Score',fontsize=11)
            plotname = conds[cond][0].capitalize()+'_'+conds[cond][1].capitalize()+'_'+freq[2:]+'_'+roi+'.png'
            plt.savefig(path2save+plotname)
            plt.clf()
            plt.close()

/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Frontal Inf Orb (L) -0.422932254801 nan 0.736904761905
Frontal Inf Orb (R) 0.379680995888 0.598806799028 0.875517045455
Fusiform (L) -0.000328988078151 0.591389080093 0.727202380952
Fusiform (R) 1.17020749914 0.404484981228 0.791145562771
Temporal Mid (L) -0.0786196953878 0.575827397974 0.768958333333
Temporal Mid (R) 1.18473776956 0.494222579644 0.789993073593
Temporal Sup (L) -0.259854559104 nan 0.72880952381
Temporal Sup (R) 0.238654777268 0.524290637011 0.822676136364
Frontal Inf Orb (L) 0.0797693771807 0.313316779039 0.892611111111
Frontal Inf Orb (R) -0.116330347686 0.129721610022 0.843166233766
Frontal Mid Orb (L) -0.452259788021 0.322301284284 0.862023809524
Frontal Mid Orb (R) 0.0778407123234 0.140363087343 0.856048917749
Fusiform (L) 0.155314375395 0.0035469333246 0.727976190476
Fusiform (R) 0.251246970829 0.303103874511 0.880570707071
Hippocampus (L) 0.141214269442 0.202560318337 0.789886904762
Hippocampus (R) -0.0733320804264 0.104879977141 0.83805988456
Temporal Inf (L) 0.

### FEATURE MATRIX
    Power, DA, Time

In [ ]:
#Create the boolean matrix to show only regions sig in at least 60% of patients
da_mat, pow_mat, time_mat = np.array([]),np.array([]),np.array([])
x_freq, x_cond = [], []
feats = ['da','time','pow_change']
groupby = 'cond'

for cond,freq in product(sorted(conds),freqs[:]): #trick to impose iteration order for conds & freqs
    x_cond.append(conds[cond][0][:2].capitalize()+'/'+conds[cond][1][:2].capitalize())
    filename60 = path_df+cond+'_Classif_'+conds[cond][0]+'_'+conds[cond][1]+'_'+freq+'_win3_patients'+str(conds[cond][2])+'.csv'
    sig60, data = pd.read_csv(filename60), np.array([])
    sig60['pow_change']= (sig60['p0']-sig60['p1'])/sig60['p1']
    gr = sig60.groupby(['s_aal','su_codes'])
    sel = gr[feats].agg(('max','min','mean'))
    da, time, rel_change = np.array([]), np.array([]), np.array([])
    for roi in roi_all:
        df_roi = sel[sel.index.get_level_values(0)==roi]
        da_val = df_roi.filter(like='da').filter(like='max').mean().values
        time_val = df_roi.filter(like='time').filter(like='mean').mean().values
        pow_min = df_roi.filter(like='pow_change').filter(like='min').mean().values
        pow_max = df_roi.filter(like='pow_change').filter(like='max').mean().values
        rel_change_val = []
        if abs(pow_min[0])>abs(pow_max[0]):
            rel_change_val = pow_min
        elif abs(pow_min[0])<abs(pow_max[0]):
            rel_change_val = pow_max
        elif abs(pow_min[0])==abs(pow_max[0]):
            rel_change_val = pow_max
        elif np.isnan(pow_min):
            rel_change_val = [np.nan]
        #print('value for',roi,pow_min, pow_max,rel_change_val, da_val, time_val)
        
        da = np.hstack((da,da_val)) if np.size(da) else da_val
        time = np.hstack((time,time_val)) if np.size(time) else time_val
        rel_change = np.hstack((rel_change,rel_change_val)) if np.size(rel_change) else rel_change_val
        #print('vector',len(da),len(time),len(rel_change))
    da_mat = np.vstack((da_mat,da)) if np.size(da_mat) else da
    time_mat = np.vstack((time_mat,time)) if np.size(time_mat) else time
    pow_mat = np.vstack((pow_mat,rel_change)) if np.size(pow_mat) else rel_change
    #print('vector',len(da_mat),len(time_mat),len(pow_mat))
da_mat,time_mat, pow_mat = da_mat.T, time_mat.T, pow_mat.T
_, idx = np.unique(x_cond,return_index=True) #to keep order
x_cond = [x_cond[i] for i in sorted(idx)]
print(da_mat.shape, time_mat.shape, pow_mat.shape)

### Plot the confusion matrix

In [ ]:
import matplotlib.pyplot as plt
from confusion_matrix import plot_confusion_matrix
%load_ext autoreload
%autoreload 2

freqnames = ['θ']#,'θ','γ2']
codes = {0:['da',plt.cm.BuPu,1],
         1:['time',plt.cm.YlGn,1],
         2:['pow_change',plt.cm.seismic,2]}
feats_mat = [da_mat, time_mat, pow_mat]

# Plot non-normalized confusion matrix
for i,mat in enumerate(feats_mat):
    print(i, len(mat))
    groups = len(np.unique(x_cond))
    print(groups, x_cond)
    nconds = len(np.unique(x_cond))
    plot_confusion_matrix(mat, xtickslabels1=freqnames*groups, xtickslabels2=x_cond, 
        ytickslabels=roi_all,cmap=codes[i][1], ylabel='Regions',size=(5,5),cbsides=codes[i][2])
    plot_name = path2save+'Matrix_recap_theta_'+codes[i][0]+'_by_'+groupby+'_'+str(nconds)+'conds_rois'+option+'.png'
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')
    plt.show()